To Do:
DO NOT GET HUNG UP ON TWEAKING - GET A WORKING MODEL FIRST
work on random forest
    count vectorizer/TFIDF ???
    TFIDF is better - switch to that only

In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import zscore
from sklearn.cluster import DBSCAN
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_classif
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
from matplotlib.pyplot import pie
%matplotlib inline
import time

import warnings
warnings.filterwarnings("ignore")

# from xgboost import XGBClassifier
# from hyperopt import hp,Trials,fmin,tpe
import pandas_profiling


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ssturtz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/ssturtz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ModuleNotFoundError: No module named 'pandas_profiling'

Import Data and Clean

In [17]:
df = pd.read_csv('Data/globalterrorismdb_0718dist.csv', encoding='latin-1', low_memory=False)

df['summary']= df['summary'].str.replace('\d+', '') #remove digits
df['summary']= df['summary'].str.replace('[^\w\s]', '')#remove punctuation
df.fillna('', inplace=True)
pd.set_option('display.max_colwidth', None) 

df.drop(df[df['summary'] == ''].index, inplace = True)

In [18]:
counts = df['gname'].value_counts()
df = df[df['gname'].isin(counts[counts > 200].index)]

New dataframe of text data to compare

In [19]:

df =df[['country_txt','weaptype1_txt','attacktype1_txt','targsubtype1_txt','motive','weapdetail','summary', 'gname']]
unk = df[df['gname'] == 'Unknown']
df.drop(df[df['gname'] == 'Unknown'].index, inplace = True)

Create holdback data

In [20]:
samp = df.sample(frac = .10)
df.drop(samp.index);




create train/test split

In [21]:
y= df['gname']
X = df.drop(columns = ['gname'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30, random_state=42)

check for common words in summary

In [23]:

X_train



,country_txt,weaptype1_txt,attacktype1_txt,targsubtype1_txt,motive,weapdetail,summary
148011,Sudan,Unknown,Hijacking,Retail/Grocery/Bakery,,,Assailants attacked a convoy of commercial vehicle between El Fashir and Tawilah North Darfur state Sudan There were no reported casualties in the attack but an unknown number of vehicles and weapons were stolen No group claimed responsibility for the incident however sources attributed the attack to the Janjaweed
145208,Niger,Explosives,Bombing/Explosion,"Military Personnel (soldiers, troops, officers, forces)",,A remote-controlled homemade landmine was used in the attack.,An explosive device detonated near military personnel in Diffa town Diffa province Niger Four people including two soldiers and two assailants were killed and one soldier was injured in the blast No group claimed responsibility for the incident however sources attributed the attack to Boko Haram
148890,Afghanistan,Firearms,Armed Assault,Police Checkpoint,,,Assailants attacked a police checkpoint in Shepano Kala locality Yosufkhel district Paktika province Afghanistan At least two assailants were killed in the ensuing clash The Taliban claimed responsibility for the attack
95534,Somalia,Explosives,Bombing/Explosion,"Government Personnel (excluding police, military)",The specific motive for the attack is unknown.,A remote controlled landmine was used in the attack.,On Friday in Mogadishu Banaadir Somalia a remotecontrolled landmine exploded near a vehicle carrying a Somali government delegation while on a security mission at Manabolyo intersection Four Somali soldiers were killed and five others were seriously injured No damage was reported and no group claimed responsibility but it was suspected that alShabaab fighters were responsible
100084,Pakistan,Unknown,Unknown,Unnamed Civilian/Unspecified,The specific motive for the attack is unknown.,Unknown weapons were used in the attack.,On Sunday afternoon in Kurram Federally Administered Tribal Areas Pakistan suspected TehrikeTalibanPakistan TTP militants attacked the Turi Bangash village of Blashkel causing a clash between them and the Turi Bangash tribe after a truce between Turi Bangash Tribes and TTP leaving six tribesmen and two Taliban militants injured Other reports claimed TTP militants were killed by that could no be confirmed The proTaliban militants are against the Turi Bangash tribes because they did not allow Talibinazation in the Upper and Lower Khurram No damages were reported and no group claimed responsibility
...,...,...,...,...,...,...,...
91970,Colombia,Explosives,Bombing/Explosion,Multinational Corporation,The specific motive for the attack is unknown.,An improvised explosive device was used in the attack.,On Monday night at about in Neiva Huila Colombia two assailants launched an improvised explosive device at an industrial zone hitting a dairy products company Alpina injuring two people and damaging the business According to police commander Flavio Mesa the same company was attacked last August because they failed to pay extortion money to the Revolutionary Armed Forces of Colombia FARC No group claimed responsibility although the police suspected the FARC was responsible
108485,Somalia,Firearms,Assassination,Radio Journalist/Staff/Facility,,Pistols were used in the attack.,Two assailants opened fire on Mohamed Nur Mohamed in Xamar Jajab district Mogadishu city Banaadir region Somalia Mohamed Nur Mohamed a journalist with BarKulan radio station was seriously injured in the attack No group claimed responsibility for the incident however officials attributed it to AlShabaab
176140,Iraq,Firearms,Armed Assault,Military Barracks/Base/Headquarters/Checkpost,,,Assailants attacked Peshmerga outposts in Zarkah Saladin Iraq In addition to an unknown number of assailant casualties at least one Peshmerga soldier was killed and two soldiers were injured in the ensuing clash The Islamic State of Iraq and the Levant ISIL claimed responsibility for the incident
71128

Random Forest

In [27]:
#vectorize summary data
new_stopwords=['unknown']
stpwrd = stopwords.words('english')
stpwrd.extend(new_stopwords)

vectorizer = CountVectorizer(lowercase=True, stop_words='english', max_features = 1000)
tfidf = TfidfVectorizer(sublinear_tf=True, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words=stpwrd )


RFX_train = vectorizer.fit(X_train)
RFX_test = vectorizer.fit(X_test)







# RFX_train = vectorizer.transform(rfd)
# RFX_test = vectorizer.transform(pred_rfd)
# RFX_blind = vectorizer.transform(blind)
# X2_train= vectorizer.transform(X2_train)
# X2_test = vectorizer.transform(X2_test)

# features = tfidf.fit_transform(X_train).toarray()
# labels = y_train
# features

array([[0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0.]])

In [ ]:
# # columns that can be removed: 
# out_columns =  ['extended', 'weapsubtype1', 'weaptype1', 'claimed',   'nperpcap', 'nwoundus', 'propvalue', 'propcomment', 'addnotes', 'INT_LOG', 'INT_IDEO', 'INT_MISC', 'INT_ANY', 'related',  'property',  'propextent','nperps', 'natlty1', 'targsubtype1','motive','dbsource','approxdate' , 'extended' , 'resolution' , 'vicinity' , 'location' , 'alternative', 'alternative_txt', 'multiple', 'attacktype2', 'attacktype2_txt' , 'attacktype3' ,'attacktype3_txt' ,'targtype2' ,	'targtype2_txt' , 	'targsubtype2'	, 'targsubtype2_txt' ,	'corp2' , 	 'target2',	'natlty2',	'natlty2_txt','targtype3',	'targtype3_txt',	'targsubtype3',	'targsubtype3_txt',	'corp3',	'target3',	'natlty3',	'natlty3_txt',	'gsubname',	'gname2',	'gsubname2',	'gname3',	'gsubname3',	'guncertain2',	'guncertain3', 'claimmode',	'claimmode_txt',	'claim2',	'claimmode2',	'claimmode2_txt','claim3',	'claimmode3',	'claimmode3_txt',	'compclaim', 'weaptype2',	'weaptype2_txt',	'weapsubtype2',	'weapsubtype2_txt',	'weaptype3', 'weapsubtype3',	'weapsubtype3_txt', 'weaptype4','weaptype4_txt',	'weapsubtype4',	'weapsubtype4_txt', 'nkillus',	'nkillter',	'nwound',	'nwoundte', 'ishostkid',	'nhostkid',	'nhostkidus',	'nhours',	'ndays',	'divert', 'kidhijcountry',	'ransom',	'ransomamt',	'ransomamtus',	'ransompaid',	'ransompaidus',	'ransomnote',	'hostkidoutcome',	'hostkidoutcome_txt', 'nreleased', 'weaptype3_txt' ]

# tdata = tdata.drop(columns = out_columns) 
# tdata.fillna('', inplace=True)